# Data Quality 

**Under Construction** 

Suppose you are confronted with a new dataset. Here we learn:

*   How to explore the new dataset   
*   How to measure the quality of the new dataset
*   How to handle low quality datapoints
*   How to handle missing data points

## Outlier Identification

*Data often contain errors that may cause incorrect results and - when used for training a machine learning model - wrong forecasting behavior.*

*We distinguish between inliers and outliers, where inliers are values inside the true feature distribution and outliers are outside.*

*Outliers should be considered with caution because sometimes data contain unusual but correct data that represent valuable information and should not be considered as error. Example: Earthquake events in seismographich records. However, outliers can also be incorrect data points, which have to be deleted from the data set or imputed with more correct values*

**2-sigma rule**: We define an **outlier** $x_k^{(i)}$ as a data point, that deviates from the mean by at least twice the standard deviation
    
$$|x_k^{(i)} - \bar{x}^{(i)} | > 2 \cdot s^{(i)}$$

with the standard deviation of the i^th feature:

$$s^{(i)} = \sqrt{\frac{1}{n-1} \sum_{k = 1}^{n} \left( x_k^{(i)} - \bar{x}^{(i)} \right)^2}$$

*If sufficient data are available and data quality is important, then suspicious data may be completely removed.*

**Note:** Also for μ-σ-standardization outliers should be removed before, they
may otherwise strongly affect feature means and standard deviations.

# Setup

In [1]:
# Common imports
import numpy as np
import os
import pandas as pd

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "data_quality"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

# Introduction

Preparing and cleaning data is a crucial aspect of building a machine-learning model. 

# First Steps with a new dataset

## Load the data

In [2]:
df = pd.read_csv('complete.csv', 
                 usecols = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 
                 low_memory=False)

Explanation:
   *    **low_memory**: lower memory use while parsing
   *    **usecols**: use subset of the columns

Familiarize with the data with certain pandas commands, like the head(), the shape(), the info() commands.

## Take a first look at the data

In [3]:
df.head(3)

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667


## Get the shape of the data

In [4]:
print('The data set consists of {} columns and {} rows.'.format(df.shape[0], df.shape[1]))

The data set consists of 88875 columns and 11 rows.


## Get the variable types

In [5]:
df.dtypes

datetime                 object
city                     object
state                    object
country                  object
shape                    object
duration (seconds)       object
duration (hours/min)     object
comments                 object
date posted              object
latitude                 object
longitude               float64
dtype: object

## Get a quick description of the data

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88875 entries, 0 to 88874
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   datetime              88875 non-null  object 
 1   city                  88679 non-null  object 
 2   state                 81356 non-null  object 
 3   country               76314 non-null  object 
 4   shape                 85757 non-null  object 
 5   duration (seconds)    88873 non-null  object 
 6   duration (hours/min)  85772 non-null  object 
 7   comments              88749 non-null  object 
 8   date posted           88875 non-null  object 
 9   latitude              88875 non-null  object 
 10  longitude             88875 non-null  float64
dtypes: float64(1), object(10)
memory usage: 7.5+ MB


## Data Type Categorization

In a next step we split the features into the data types: numerical, categorical and datetime objects. For both of them we develop a data summary, perform graphical representations and try to understand the interavtions among the features of the same data type as well as from the other data type.

**Data summary**: number of missing values, number of unique values, descriptice statistics: mean, mode, median, standard deviation, number and name of different categories.

**Graphical representation**: Seaborn package is especially well suited. It offers nice informative statistical graphics. Outliers can be detected with boxplots, more information on the underlying data via distribution plots. For categorical variables a first graphical representation can be a bar plot.
Correlation among numerical variables can be detetected via a scatter plot and correlation coefficients. For correlations among categorical variables a chi2-test, the crosstabl function and bar plots with color encoding are helpful. For correlations of numerical and categorical variables, boxplots are helpful.

## Datetime variables

In [8]:
df['datetime']     = pd.to_datetime(df.datetime.str.replace('24:','00:'))
df['datetime_str'] = df['datetime'].dt.strftime('%Y-%m-%d %H:%M') 

In [10]:
df[['datetime', 'datetime_str']].dtypes

datetime        datetime64[ns]
datetime_str            object
dtype: object

In [11]:
df.dtypes

datetime                datetime64[ns]
city                            object
state                           object
country                         object
shape                           object
duration (seconds)              object
duration (hours/min)            object
comments                        object
date posted                     object
latitude                        object
longitude                      float64
datetime_2                      object
dtype: object

In [11]:
print('First timestamp of dataset is {} and last timestamp is {}.'.format(min(df.datetime), max(df.datetime)))

First timestamp of dataset is 1906-11-11 00:00:00 and last timestamp is 2014-05-08 18:45:00.


### Search for duplicated dates

In [23]:
dfBySize = (df.groupby('datetime').size() > 1).astype(float).to_frame().reset_index()
dfBySize.columns = ['datetime', 'size']
dfBySize[dfBySize['size'] > 0]

,datetime,size
41,1943-08-15 00:00:00,1.0
45,1944-01-01 12:00:00,1.0
59,1945-06-01 12:00:00,1.0
97,1947-07-01 20:00:00,1.0
110,1947-07-15 15:00:00,1.0
...,...,...
75980,2014-05-03 22:00:00,1.0
75984,2014-05-03 22:30:00,1.0
76021,2014-05-06 22:00:00,1.0
76024,2014-05-06 23:00:00,1.0


In [25]:
df[df.datetime == '1944-01-01 12:00:00']

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,datetime_str
11941,1944-01-01 12:00:00,san diego,ca,us,cigar,180,3 minutes,A sighting of one silver or white enlongated c...,7/25/2004,32.7152778,-117.156389,1944-01-01 12:00
11942,1944-01-01 12:00:00,wilderness,wv,NaN,disk,1814400,3 weeks,Two related reports of possible predecessor to...,7/8/2004,0,0.000000,1944-01-01 12:00


In [13]:
(df.groupby('datetime').size() > 1).value_counts()

False    67370
True      8670
dtype: int64

In [14]:
(df.groupby('datetime_2').size() > 1).value_counts()

False    67370
True      8670
dtype: int64

In [27]:
df.datetime.isin(df.groupby('datetime').size() > 1).index#.to_list()

RangeIndex(start=0, stop=88875, step=1)

In [16]:
df_dupl = (df.groupby('datetime_2').size() > 1).to_frame()

In [17]:
df_dupl.columns = ['# of entries']

In [18]:
df_dupl[df_dupl['# of entries'] == True].index

Index(['1943-08-15 00:00', '1944-01-01 12:00', '1945-06-01 12:00',
       '1947-07-01 20:00', '1947-07-15 15:00', '1947-07-15 21:00',
       '1948-06-01 00:00', '1949-09-15 21:00', '1950-06-15 20:00',
       '1950-06-30 17:30',
       ...
       '2014-05-01 21:15', '2014-05-01 21:30', '2014-05-01 23:00',
       '2014-05-02 22:00', '2014-05-03 21:00', '2014-05-03 22:00',
       '2014-05-03 22:30', '2014-05-06 22:00', '2014-05-06 23:00',
       '2014-05-07 20:30'],
      dtype='object', name='datetime_2', length=8670)

In [19]:
df[df.datetime == '1943-08-15 00:00']#.shape[0]

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,datetime_2
73312,1943-08-15,unknown,la,NaN,light,900,10-15 minutes,1942 Louisiana midnight light like a...,2/14/2008,37.245443,-107.827839,1943-08-15 00:00
73313,1943-08-15,unknown,la,NaN,unknown,720,10-12 minutes,White light at midnight traveling into outer s...,1/30/2014,37.245443,-107.827839,1943-08-15 00:00


In [28]:
# %timeit 
df_dupl = df.loc[df.datetime.isin(df.groupby('datetime').size() > 1).index.to_list()]

In [30]:
df_dupl.head(12)

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,datetime_str
0,1949-10-10 20:30:00,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111,1949-10-10 20:30
1,1949-10-10 21:00:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082,1949-10-10 21:00
2,1955-10-10 17:00:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667,1955-10-10 17:00
3,1956-10-10 21:00:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833,1956-10-10 21:00
4,1960-10-10 20:00:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611,1960-10-10 20:00
5,1961-10-10 19:00:00,bristol,tn,us,sphere,300,5 minutes,My father is now 89 my brother 52 the girl wit...,4/27/2007,36.5950000,-82.188889,1961-10-10 19:00
6,1965-10-10 21:00:00,penarth (uk/wales),NaN,gb,circle,180,about 3 mins,penarth uk circle 3mins stayed 30ft above m...,2/14/2006,51.434722,-3.180000,1965-10-10 21:00
7,1965-10-10 23:45:00,norwalk,ct,us,disk,1200,20 minutes,A bright orange color changing to reddish colo...,10/2/1999,41.1175000,-73.408333,1965-10-10 23:45
8,1966-10-10 20:00:00,pell city,al,us,disk,180,3 minutes,Strobe Lighted disk shape object observed clos...,3/19/2009,33.5861111,-86.286111,1966-10-10 20:00
9,1966-10-10 21:00:00,live oak,fl,us,disk,120,several minutes,Saucer zaps energy from powerline as my pregna...,5/11/2005,30.2947222,-82.984167,1966-10-10 21:00


## Create a test set

### Random Sampling

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
train_set, test_set = train_test_split(df, test_size = 0.2, random_state = 42)

In [25]:
train_set.head(2)

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,datetime_2
25764,2006-01-27 01:00:00,armagh (republic of ireland),NaN,NaN,unknown,1200,20 minutes,the lights in our town were too bright for any...,2/14/2006,0,0.000000,2006-01-27 01:00
53455,2011-06-12 22:56:00,hudson falls,ny,us,other,600,10 minutes,Spiral ufo seen in hudson falls.,6/20/2011,43.3005556,-73.586389,2011-06-12 22:56


This is simple **random sampling**.

### Stratified Sampling

In [26]:
df.city.value_counts()

seattle                   570
phoenix                   488
portland                  399
las vegas                 395
los angeles               375
                         ... 
raynham (northeast of)      1
castries (st. lucia)        1
harris ranch                1
ledbetter                   1
beaverton/tigard            1
Name: city, Length: 22018, dtype: int64